In [1]:
import src.decorrelate as cluster_model
MODEL_NAME = 'EleutherAI/pythia-70m'
DATASET_NAME = 'NeelNanda/pile-10k'

N_DIMS = 512
SEED = 69_420
 
DEBUG = False
 
if DEBUG:
    N_DATASIZE = 300
    N_BLOCKS = 12
    STRING_SIZE_CUTOFF = 200
else:
    # It gets killed aroun 1_800 idk why. Maybe we have a problem with token truncation somewhere
    N_DATASIZE = 3_000
# 
    # N_CLUSTERS_MIN = int(0.5 * N_DIMS)
    # N_CLUSTERS_MAX = 10 * N_DIMS
    # TODO: DEL ME
    N_BLOCKS = 6
    STRING_SIZE_CUTOFF = 1_200

params = cluster_model.InterpParams(
	lattice_params=cluster_model.LatticeParams(
		top_layer_idx = -1,
        max_n_parents = 4
	),
    # quantization='4bit',
	seed=SEED,
    n_datasize=N_DATASIZE,
    n_blocks=N_BLOCKS,
    model_name=MODEL_NAME,
	model_n_dims=N_DIMS,
    dataset_name=DATASET_NAME,
    string_size_cutoff=STRING_SIZE_CUTOFF,
    quantization='4bit'
)

decomp = cluster_model.Decomposer(params)
decomp.load(reload=True)

/home/lev/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating decomposer with parameter data hash cache/data-05e441406567ff2be4dc9c51e891918f5747747a/start.pkl
Creating decomposer with parameter lattice hash cache/correlation-d2e9a74f412a5e15e0fcde6056db6a751baf0149/start.pkl
Created dataset


/home/lev/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Finished and saving to file

Got embeddings
Computing internal correlations for layer 0
(1024, 507306) (1024, 507306)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/lev/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_225415/1769244142.py", line 41, in <module>
    decomp.load(reload=True)
  File "/home/lev/code/research/ai/lattice_cluster_paths/src/decorrelate.py", line 251, in load
    self.model, self.dataset, self.layers, params=self.params, use_save=not reload)
  File "/home/lev/code/research/ai/lattice_cluster_paths/src/decorrelate.py", line 194, in correlate_internal_layer
  File "/home/lev/code/research/ai/lattice_cluster_paths/src/utils.py", line 292, in pairwise_correlation_metric
    return pairwise_signaling(A, B)
  File "/home/lev/code/research/ai/lattice_cluster_paths/src/utils.py", line 249, in pairwise_signaling
    signaling[i, j] = p_i_and_j.sum() / p_i_j_min
  File "/home/lev/.local/lib/python3.10/site-packages/numpy/core/_methods.py", line 49, in _sum
    retu

## Lets just look at the highest cliques

In [ ]:
decomp.internal_correlations[1]

## Build Up a "Concept" Lattice Using Graph Restrictions

> TODO: this is for latter...

In [ ]:
import src.graph as graph
import networkx as nx

LAYER = 3

G = graph.graph_from_correlations(decomp.internal_correlations[LAYER])
MAX_CLIQUE_SIZE = 1_000

GG = graph.sparsify_weighted_graph(G, degree_upperbound=400)
i = nx.find_cliques(GG)
clique = next(i)

In [ ]:
import importlib
import src.utils as utils
importlib.reload(utils)

# decomp.tune_clique((0.0, clique), 3)

In [ ]:
# import networkx as nx

# test_at = 100
# GG = graph.sparsify_weighted_graph(G, degree_upperbound=test_at)

# cliques = []

# for node in range(N_DIMS * 2):
#     if node % 50 == 0:
#         print("On node", node)
#     c_it = nx.find_cliques(GG, nodes=[node])
#     c_in_curr = []
#     for c in c_it:
#         c_in_curr.append(c)
#         if len(c_in_curr) > 50:
#             break
#     cliques += c_in_curr
#         # if len(cliques) > 10_000:
#         #     break

## Score a cell

In [ ]:
decomp.internal_correlations[0]

In [ ]:
import src.decorrelate as decc
import src.kernel as kernel
importlib.reload(decc)
importlib.reload(utils)
importlib.reload(kernel)

LAYER = 2
NEURON = 20

# TODO: we **need** to do something like curr-clique removal so we get more interesting cliques...
# Also the per-neuron thing makes no sense...
decomp.scores_for_neuron(LAYER, NEURON, degree_upperbound=400)